In [7]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import iDEA
import pickle
from scipy.integrate import simpson

l = 5
points = 750
x = np.linspace(0, l, points)
dx = x[1] - x[0]
v_ext = np.zeros(len(x))
v_int = np.zeros([len(x), len(x)])
pib_double = iDEA.system.System(x, v_ext, v_int, electrons = "ud")
states = {}

for i in range(10):
    try:
        with open(f"state750_{i}.pkl", "rb") as file:
            states[i] = pickle.load(file)
    except FileNotFoundError:
        print(f"File state_{i}.pkl not found.")
    except Exception as e:
        print(f"An error occurred while loading state_{i}.pkl: {e}")

In [9]:
# ground_state = states[0]
# n = iDEA.observables.density(pib_double, ground_state)

# ground_analytic = (2/np.sqrt(2))*wave_function(l, 0, x_1) * wave_function(l, 0, x_2)
# n_analytic = (np.abs(ground_analytic))**2
# plt.plot(x, n, x, n_analytic)

In [10]:
# def state_analysis(states: dict, l, n1: int, n2: int, k: int):
#     x = np.linspace(0, l, 750)
#     dx = x[1] - x[0]
#     x_1 = np.linspace(0, l, 750)
#     x_2 = np.linspace(0, l, 750)
#     x_1[0] = x_1[1] - dx
#     x_1[-1] = x_1[-2] + dx
#     x_2[0] = x_2[1] - dx
#     x_2[-1] = x_2[-2] + dx
#     X_1, X_2 = np.meshgrid(x_1, x_2)

#     state = states[k]
#     pd_approx = (np.abs(state.space.real))**2

#     state_analytic = wave_function((l+2*dx), n1, X_1) * wave_function((l+2*dx), n2, X_2)
#     pd_analytic = (np.abs(state_analytic))**2

#     fig, (ax1, ax2) = plt.subplots(1, 2)

#     ax1.imshow(pd_analytic, cmap="seismic", vmax=np.max(pd_analytic), vmin=-np.max(pd_analytic))
#     ax1.set_title("Probability density (analytic)")

#     ax2.imshow(pd_approx, cmap="seismic", vmax=np.max(pd_approx), vmin=-np.max(pd_approx))
#     ax2.set_title("Probability density (approx)")
#     plt.tight_layout()
#     plt.show()

#     print(np.allclose(pd_approx, pd_analytic))

    

In [55]:
def wave_function(l, n, X):
    return np.sqrt(2/l) * np.sin((np.pi) * X * (n+1) / l)

def states_density_comparison(states: dict, s: iDEA.system.System, l, points):
    # Initialize system
    x = s.x
    x_prime = x
    dx = s.dx
    y = np.linspace(-dx, l+dx, points+2)

    orbital_space = []
    analytic_space_1 = []
    analytic_space_2 = []
    state_index_list = [[0, 0], [0, 1], [1, 0], [1, 1], [0, 2], [2, 0], [1, 2], [2, 1], [0, 3], [3, 0]]
    for key, value in states.items():
        n = iDEA.observables.density(s, value)

        index_k = state_index_list[key][0]
        index_j = state_index_list[key][1]
        # integrand_1 = (np.abs(wave_function(l, index_j, x_prime))) ** 2
        # integral = simpson(integrand_1, x=x_prime)

        # many_body = np.sqrt(1/2) * (wave_function(l, index_k, x)*wave_function(l, index_j, x) + wave_function(l, index_k, x_prime)*wave_function(l, index_j, x_prime))
        # integrand_2 = np.abs(many_body)**2
        # integral = simpson(integrand_2, x=x_prime)
        # pd_analytic_1 = abs(wave_function(l, state_index_list[key][0], x) * wave_function(l, state_index_list[key][1], x)) ** 2

        # print(integral)

        pd_analytic_1 = (np.abs(wave_function(l, index_k, x)))**2 + (np.abs(wave_function(l, index_j, x)))**2
        pd_analytic_2 = abs(wave_function((l+2*dx), state_index_list[key][0], y) * wave_function((l+2*dx), index_j, y)) ** 2
        pd_analytic_2 = pd_analytic_2[1:-1]
        
        orbital_space.append(n)
        analytic_space_1.append(pd_analytic_1)
        analytic_space_2.append(pd_analytic_2)

    # Ensure the directory for saving images exists
    # output_dir = 'frames'
    # if not os.path.exists(output_dir):
    #     os.makedirs(output_dir)
    # Define the function to plot the array
    def plot_array(k):
        plt.figure(figsize=(10, 6))
        plt.plot(x, orbital_space[k], "black")
        plt.plot(x, analytic_space_1[k], "b--")
        plt.plot(y[1:-1], analytic_space_2[k], "r--")
        plt.title(f'State {k}')
        plt.xlabel('x')
        plt.ylabel('y')
        plt.grid(True)
        plt.legend(["Approximate prob. density", "Analytical prob. density (l)", "Analytical prob. density (l+2*dx)"])
       
        # Check if the arrays are approximately equal
        approx_close_1 = np.allclose(orbital_space[k], analytic_space_1[k], rtol=1e-05, atol=1e-08)
        approx_close_2 = np.allclose(orbital_space[k], analytic_space_2[k], rtol=1e-05, atol=1e-08)
        
        print(f"Is approximate close to analytic 1 for state {k}?: {approx_close_1}")
        print(f"Is approximate close to analytic 2 for state {k}?: {approx_close_2}")

        # plt.savefig(os.path.join(output_dir, f'frame_{k:03d}.png'))
        plt.show()
    
    # for k in range(len(orbital_space)):
    #     plot_array(k)

    # Create a slider widget
    slider = widgets.IntSlider(value=0, min=0, max=len(orbital_space) - 1, step=1, description='k:')
    interactive_plot = widgets.interactive(plot_array, k=slider)
    
    # Display the interactive plot
    display(interactive_plot)
    # plt.plot(x, (analytic_space_2[0] - orbital_space[0]))
    # plt.show()

In [56]:
states_density_comparison(states, pib_double, l, points)

interactive(children=(IntSlider(value=0, description='k:', max=9), Output()), _dom_classes=('widget-interact',…